# CLASIFICADOR DE IMAGENES

-------------------------------------------------------

primero hacerlo y probarlo como lo hacen en algun ejemplo de internet

despues recolectar datos (usando lo que paso willy + Google formularies) para hacer el clasificador

usar esa red pero con los datos recolectados

Hacer crossvalidation o algo asi para ajustar las variables

-------------------------------------------------------

In [1]:

import sys
import os


from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

#Funciones y modelos que vamos a usar de KEras 
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D

In [2]:
#Matar las sesiones anteriores de Keras
from tensorflow.python.keras import backend as K 
K.clear_session()

In [3]:
# Defino ruta a los datos que voy a usar
data_entrenamiento = './data/entrenamiento'
data_validacion = './data/validacion'

Parametros de la red y de entrenamiento

In [5]:
#epocas, son como las vueltas que da
epocas=5 #20

#tamaño de imagenes, 150x150 pixels
longitud, altura = 150, 150 

#cantidad de imagenes que va procesar en cada epoca (tamaño de lotes)
batch_size = 32

#numero de veces que se va a procesar la informacion en cada epoca
pasos = 100 #1000

#numero de veces que se va a validar la informacion en cada epoca
validation_steps = 300

#Profundidad de las dos capas de convolucion
filtrosConv1 = 32
filtrosConv2 = 64

#tamaño de los filtros que vamos a usar en las convoluciones
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)

#Tipo de clases, en este ejemplo --> gatos, perros y gorillas
clases = 3

#"Learning Rate" --> tamaño de ajuste a la red para acercarse a la mejor solucion
lr = 0.0004



Preparamos nuestras imagenes

In [6]:
#defino generadores de datos

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255, #rescalar las imagenes --> en vez de 0 a 255, lo cambio para que los valores esten entr 0 y 1
    shear_range=0.2, # va a inclinar las imagenes aleatoriamente
    zoom_range=0.2, # va a ser zoom aleatorios
    horizontal_flip=True) #invierte la imagen aleatoriamente

test_datagen = ImageDataGenerator(
    rescale=1. / 255) # para la validacion no queremos que modifique la imagen (solo el rescalado)

#Extraigo imagenes

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento, # la ruta a los datos de entrenamiento
    target_size=(altura, longitud), # lo que definimos antes
    batch_size=batch_size, # tamaño de lotes que ya definimos
    class_mode='categorical') #clasificacion categorica --> es decir que es por clases

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')


Found 999 images belonging to 3 classes.
Found 2043 images belonging to 3 classes.


Definiendo parametros estructurales de la red neuronal

In [7]:
#Red secuencial, varias capas apiladas
cnn = Sequential()

#Primera Capa --> convolucion
cnn.add(Convolution2D(   #defino la primera capa --> de convolucion
    filtrosConv1, # 32 filtros
    tamano_filtro1, 
    padding ="same", 
    input_shape=(longitud, altura, 3), # lo que va a entrar (obs, el 3 es porque es rgb)
    activation='relu')) #relu es la funcion de activacion de la neurona

#Segunda capa --> Max pooling
cnn.add(MaxPooling2D(pool_size=tamano_pool))  #agrupa pixeles y agarra el valor del pixel mas grande

#Tercera capa --> convolucion
cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))


#Cuarta capa --> Max pooling
cnn.add(MaxPooling2D(pool_size=tamano_pool)) #agrupa pixeles (en este caso era de 2x2) y agarra el valor del pixel mas grande


cnn.add(Flatten()) # aplana la imagen, 1 sola dimension


#Quinta capa-->  Dense
cnn.add(Dense(
    256, #256  neuronas "normales"
    activation='relu')) #con activacion relu


cnn.add(Dropout(0.5)) # durate el entrenamiento le apago el 0,5 de las neuronas --> esto es para evitar el overfitting


#Sexta capa -->Dense --> ULTIMA CAPA --> tiene 3 neuronas porque son 3 clases
cnn.add(Dense(clases, activation='softmax')) # la funcion softmax te va a decir que probabilidad de que sea cada clase
                                            # Softmax es utilizada en la capa final de los clasificadores


Entrenamiento

In [ ]:
# Este es el que me va a ir ajustando el error para ir entrenando la red    
cnn.compile(loss='categorical_crossentropy', #te dice que tan bien le esta yendo
            optimizer=optimizers.Adam(lr=lr), #optimizador Adam --> ajusta los parametros
            metrics=['accuracy']) #lo que trata de mejorar, en este caso el porcentaje de imagenes que acerto --> accuracy

# El proceso de entrenamiento, con epocas, lotes, etc
cnn.fit_generator(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps)

Epoch 1/5


Guardar Modelo Entrenado 

In [2]:
import h5py
target_dir = './modelo/' # geero carpeta "modelo"

if not os.path.exists(target_dir):
  os.mkdir(target_dir)

cnn.save('./modelo/modelo.h5') # guarda la estructura del modelo en si
cnn.save_weights('./modelo/pesos.h5') # guarda los pesos de las variables

NameError: name 'cnn' is not defined

# --------------------------------------------------------------

In [26]:
import numpy as np
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.models import load_model

longitud, altura = 150, 150
modelo = './modelo/modelo.h5'
pesos_modelo = './modelo/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

ImportError: `load_model` requires h5py.

In [27]:
def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x)
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Perro")
  elif answer == 1:
    print("pred: Gato")
  elif answer == 2:
    print("pred: Gorila")

  return answer

In [30]:
import cv2
abc= cv2.imread('./data/abc.jpg',1)

ImportError: No module named 'cv2'

In [59]:
predict('./data/entrenamiento/gorila/20.jpeg')

pred: Gorila


2